# Preprocessing brain MRI for quantitative imaging analysis

## resampling, coregistration, N4 bias correction, and skull stripping

## Before using MONSTR

### Setting PATH variable
**The environment PATH MUST be set in the main terminal, not otherwise like in a sub-terminal in the Jupyterlab**

#### ANTs
export ANTSPATH=/opt/ANTs/bin/

export PATH=${ANTSPATH}:$PATH

#### FSL
FSLDIR=/usr/local/fsl
. ${FSLDIR}/etc/fslconf/fsl.sh

PATH=${FSLDIR}/bin:${PATH}

export FSLDIR PATH


In [3]:
import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

sourceDir = '/media/chansik/Samsung_T5/test_nifti'
targetDir = '/media/chansik/Samsung_T5/test_stripped'
outputPath = '/home/chansik/MONSTR/output'

T1_BB = sorted([i for i in os.listdir(sourceDir) if 'BB' in i])
T1_WB = sorted([i for i in os.listdir(sourceDir) if 'WB' in i])
T2 = sorted([i for i in os.listdir(sourceDir) if 'T2' in i])
alreadyDoneList = []

for (BB, T2, WB) in zip(T1_BB, T2, T1_WB):
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = BB.split('_')[0] 
    print((BB, T2, WB))
     print("<<< Started working on the following case: ", mrn, " >>>")
        
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")

/home/chansik/MONSTR
('10012330_BB.nii.gz', '10012330_T2.nii.gz', '10012330_WB.nii.gz')
('10013011_BB.nii.gz', '10013011_T2.nii.gz', '10013011_WB.nii.gz')
('10019513_BB.nii.gz', '10019513_T2.nii.gz', '10019513_WB.nii.gz')
('10021572_BB.nii.gz', '10021572_T2.nii.gz', '10021572_WB.nii.gz')
('10021589_BB.nii.gz', '10021589_T2.nii.gz', '10021589_WB.nii.gz')
('10023950_BB.nii.gz', '10023950_T2.nii.gz', '10023950_WB.nii.gz')
('10030605_BB.nii.gz', '10030605_T2.nii.gz', '10030605_WB.nii.gz')
('10048545_BB.nii.gz', '10048545_T2.nii.gz', '10048545_WB.nii.gz')
('10051844_BB.nii.gz', '10051844_T2.nii.gz', '10051844_WB.nii.gz')
('167043_BB.nii.gz', '167043_T2.nii.gz', '167043_WB.nii.gz')
('452824_BB.nii.gz', '452824_T2.nii.gz', '452824_WB.nii.gz')


In [1]:
######################
##########metastasis for PYW ###########3


import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/scmlsg/MONSTR'
%cd {workingDir} 

# set paths for source and target
sourceDir = '/media/scmlsg/Samsung_T5/test_nifti'
targetDir = '/media/scmlsg/Samsung_T5/test_stripped'
outputPath = '/home/scmlsg/MONSTR/output'
##########################################################################################

alreadyDoneList = []

# file names MUST be BB.nii, WB.nii, T2.nii, FLAIR.nii, and ADC.nii

for patient in os.listdir(sourceDir):
    patientDir = os.path.join(sourceDir, patient)
    
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if patient in os.listdir(targetDir):
        print(patient, 'has already been processed!')
        alreadyDoneList.append(patient)
        continue
    
    print("<<< Started working on the following case: ", patient, " >>>")
    
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")
    
    # 2. N4 bias correction
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n2) N4 bias field correction: ", end="")
    for file in pathList:
        !N4BiasFieldCorrection -d 3 -i {file} -o {file} -s 3 -c [50x50x50x50,0.00001] -b [150]
        print(file, end="  ")
  
    # 3. Co-registration
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n3) Co-registration with CE-T1 images as fixed, reference images: ", end="")
    for file in ["T2.nii", "WB.nii"]:
        newName = file.split(".")[0] + "_registered.nii"
        !antsRegistration -d 3 -r [BB.nii, {file}, 1] \
                          -m mattes[BB.nii, {file}, 1, 32, regular, 0.1 ] \
                          -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                          -m mattes[BB.nii, {file} , 1 , 32, regular, 0.1] \
                          -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {newName}] -v 0
        print(file, end="  ")
    shutil.copyfile("BB.nii", "BB_registered.nii")
 
    # 4. MONSTR skull stripping
    t1 = "WB_registered.nii"
    t2 = "T2_registered.nii"
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

    # Copy the relevant files (brain Mask and processed images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    os.makedirs(os.path.join(targetDir, patient))
    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = patient + '/brain_mask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
    shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
    shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))
 
    # 5. Clean the temporarily created files and directories
    shutil.rmtree(createdDir)

/home/scmlsg/MONSTR
<<< Started working on the following case:  167043  >>>
1) Resampling to 1-mm isovoxel images: /home/scmlsg/MONSTR/BB.nii  /home/scmlsg/MONSTR/T2.nii  /home/scmlsg/MONSTR/WB.nii  
2) N4 bias field correction: /home/scmlsg/MONSTR/BB.nii  /home/scmlsg/MONSTR/T2.nii  /home/scmlsg/MONSTR/WB.nii  
3) Co-registration with CE-T1 images as fixed, reference images: T2.nii  WB.nii  I found ANTs installation at /opt/ANTs/bin 
Default number of atlases: 4 
Number of processors is not mentioned. Using all available 6 processors. 
Atlas directory : /home/scmlsg/MONSTR/TBI_atlas
T1 : /home/scmlsg/MONSTR/WB_registered.nii
T2 : /home/scmlsg/MONSTR/T2_registered.nii
Output directory : /home/scmlsg/MONSTR/output 
Temporary files will be kept.
Images are assumed to be registered. T2/PD/FLAIR will not be registered again to the T1. 
Working directory : /home/scmlsg/MONSTR/output/WB_registered.8U9o
cd /home/scmlsg/MONSTR/output/WB_registered.8U9o
** AFNI converts NIFTI_datatype=64 (FLOAT

Using double precision for computations.
Input scalar image: /home/scmlsg/MONSTR/TBI_atlas/atlas3_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 22 SEC 
Using double precision for computations.
Input scalar image: /home/scmlsg/MONSTR/TBI_atlas/atlas3_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas3_T2_ANTS.nii
Using double precision for computations.
Input scala

Could not find library, using working directory /home/scmlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
Writing file to: /home/scmlsg/mipav/plugins/libjinput-linux64.so
cli	####################################################################
cli	Interpretting command line arguments
cli	####################################################################
edu.jhu.ece.iacl.plugins.segmentation.skull_strip.MedicAlgorithmSmoothBrainMask
	inOriginal = t1.nii
	inBrain = t1_MultiModalStripMask.nii
	inLevel = <default> = 0.5
	inMax = 0.3
	inTranslation = <default> = 0
	outSmoothed = <default> = null
	outSmooth = /home/scmlsg/MONSTR/output/WB_registered.8U9o/t1_MultiModalStripMask_smooth.nii
	outMasked = <default> = Masked Brain Volume
	outProc = /home/scmlsg/MONSTR/output/WB_registered.8U9o
cli	####################################################################
edu.jhmi.rad.medic.utilities.CubicVolumeCropper	Cropping 

N4BiasFieldCorrection -d 3 -i t2.nii -o t2.nii -s 3 -c [ 50x50x50x50,0.00001] -b [ 150]
AntsExample.sh t1.nii /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii fastfortesting atlas1_T1_ANTS.nii 6
antsRegistration -d 3 -r [ t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii ,1]  -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii , 1 , 32, regular, 0.1 ] -t translation[ 0.1 ] -c [100x50x25,1.e-8,20 ]  -s 4x2x1vox  -f 6x4x2 -l 1 -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii , 1 , 32, regular, 0.1 ] -t rigid[ 0.1 ] -c [100x50x25,1.e-8,20 ]  -s 4x2x1vox  -f 3x2x1 -l 1 -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii , 1 , 32, regular, 0.1 ] -t affine[ 0.1 ] -c [100x50x25,1.e-8,20 ]  -s 4x2x1vox  -f 3x2x1 -l 1 -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii , 0.5 , 32 ] -m cc[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas1_T1.nii , 0.5 , 4 ] -t SyN[ .20, 3, 0 ] -c [ 100x1x0,0.0001,4 ] -s 1x0.5x0vox  -f 4x2x1 -l 1 -u 1 -z 1 -o [ atlas1_T

Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
antsRegistration -d 3 -r [ t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii ,1]  -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii , 1 , 32, regular, 0.1 ] -t translation[ 0.1 ] -c [100x50x25,1.e-8,20 ]  -s 4x2x1vox  -f 6x4x2 -l 1 -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii , 1 , 32, regular, 0.1 ] -t rigid[ 0.1 ] -c [100x50x25,1.e-8,20 ]  -s 4x2x1vox  -f 3x2x1 -l 1 -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii , 1 , 32, regular, 0.1 ] -t affine[ 0.1 ] -c [100x50x25,1.e-8,20 ]  -s 4x2x1vox  -f 3x2x1 -l 1 -m mattes[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii , 0.5 , 32 ] -m cc[  t1.nii, /home/scmlsg/MONSTR/TBI_atlas/atlas4_T1.nii , 0.5 , 4 ] -t SyN[ .20, 3, 0 ] -c [ 100x1x0,0.0001,4 ] -s 1x0.5x0vox  -f 4x2x1 -l 1 -u 1 -z 1 -o [ atlas4_T1_ANTS ] -v 0 --float
Using double precis

Could not find library, using working directory /home/scmlsg/jist-lib
cli	####################################################################
cli	Initializing MIPAV/JIST Framework
cli	####################################################################
cli	Interpretting command line arguments
cli	####################################################################
edu.jhu.ece.iacl.plugins.segmentation.skull_strip.MedicAlgorithmSmoothBrainMask
	inOriginal = t1.nii
	inBrain = t1_MultiModalStripMask.nii
	inLevel = <default> = 0.5
	inMax = 0.3
	inTranslation = <default> = 0
	outSmoothed = <default> = null
	outSmooth = /home/scmlsg/MONSTR/output/WB_registered.2VTx/t1_MultiModalStripMask_smooth.nii
	outMasked = <default> = Masked Brain Volume
	outProc = /home/scmlsg/MONSTR/output/WB_registered.2VTx
cli	####################################################################
edu.jhmi.rad.medic.utilities.CubicVolumeCropper	Cropping Image...
Module Failed in an Ungraceful Manner. Please contact th

In [7]:
sourceDir = '/media/chansik/Samsung_T5/test'
targetDir = '/media/chansik/Samsung_T5/mets_stripped'
outputPath = '/home/chansik/MONSTR/output'

# Copy the relevant files (brain Mask and processed images)
createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])
patient = '1234'

os.makedirs(os.path.join(targetDir, patient))
for file in os.listdir(createdDir):
    if file == 'atlas4_brainmask_ANTS.nii.gz':
        newFileName = patient + '/brain_mask.nii.gz'
        shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))
    
shutil.copyfile("WB_registered.nii", os.path.join(targetDir, patient, "WB_registered.nii"))
shutil.copyfile("BB_registered.nii", os.path.join(targetDir, patient, "BB_registered.nii"))
shutil.copyfile("T2_registered.nii", os.path.join(targetDir, patient, "T2_registered.nii"))

# Clean the temporarily created files and directories
#os.remove(os.path.join(workingDir, t1))
#os.remove(os.path.join(workingDir, t2))
#shutil.rmtree(createdDir)


'/media/chansik/Samsung_T5/mets_stripped/1234/T2_registered.nii'

In [ ]:
t1 = "BB_registered.nii"
t2 = "T2_registered.nii"

!./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

In [ ]:
 !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas --reg

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)

In [4]:
######################
##########test ###########3


import os
import shutil

# we'll work in the MONSTR directory
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

# set paths for source and target
sourceDir = '/media/chansik/Samsung_T5'
targetDir = '/media/chansik/SC_research_2019/GBM-Surv/Seg_results'
##########################################################################################

alreadyDoneList = []

# file names MUST be CE-T1.nii, UE-T1.nii, T2.nii, FLAIR.nii, and ADC.nii

for patient in os.listdir(sourceDir):
    patientDir = os.path.join(sourceDir, patient)
    
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if patient in os.listdir(targetDir):
        print(patient, 'has already been processed!')
        alreadyDoneList.append(patient)
        continue
    
    print("<<< Started working on the following case: ", patient, " >>>")
    
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    pathList = []
    for file in os.listdir(patientDir):
        shutil.copyfile(os.path.join(patientDir, file), os.path.join(workingDir, file))
        pathList.append(os.path.join(workingDir, file))
    
    # 1. Resampling onto 1mm x 1mm x 1mm images 
    print("1) Resampling to 1-mm isovoxel images: ", end="")
    for file in pathList:
        !ResampleImage 3 {file} {file} 1x1x1
        print(file, end="  ")
    
    # 2. N4 bias correction
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n2) N4 bias field correction: ", end="")
    for file in pathList:
        !N4BiasFieldCorrection -d 3 -i {file} -o {file} -s 3 -c [50x50x50x50,0.00001] -b [150]
        print(file, end="  ")
  
    # 3. Co-registration
    # by using ANTs, using the same parameters as the default one used by MONSTR
    print("\n3) Co-registration with CE-T1 images as fixed, reference images: ", end="")
    for file in ["UE-T1.nii", "T2.nii", "FLAIR.nii", "ADC.nii"]:
        newName = file.split(".")[0] + "_registered.nii"
        !antsRegistration -d 3 -r [CE-T1.nii, {file}, 1] \
                          -m mattes[CE-T1.nii, {file}, 1, 32, regular, 0.1 ] \
                          -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                          -m mattes[CE-T1.nii, {file} , 1 , 32, regular, 0.1] \
                          -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {newName}] -v 0
        print(file, end="  ")
    shutil.copyfile("CE-T1.nii", "CE-T1_registered.nii")
 

/home/chansik/MONSTR


FileNotFoundError: [Errno 2] No such file or directory: '/media/chansik/SC_research_2019/GBM-Surv/Seg_results'

In [ ]:
    # check if preprocessing has already been done
    # if so, skip the patient and move onto the next patient
    if mrn in [i.split('_', 1)[0] for i in os.listdir(targetDir)]:
        print(mrn, 'has already been processed!')
        alreadyDoneList.append(mrn)
        continue
       
    # 0. Copy the current T1 and T2 files onto the working directory 
    #    (to avoid 'sudo' problem in trying to write a file on an external storage)
    shutil.copyfile(os.path.join(sourceDir, t1), os.path.join(workingDir, t1))
    shutil.copyfile(os.path.join(sourceDir, t2), os.path.join(workingDir, t2))
    
    # 1. Resampling of CE-T1-weighted images onto 1mm x 1mm x 1mm images 
    #   (as some T1 images are 2D)
    !ResampleImage 3 {t1} {t1} 1x1x1 
    
    # 2. Skull stripping by MONSTR algorithm following 
    #          1) rigid coregistration of T2 onto T1 (ANTs)
    #          2) N4 bias correction (ANTs)
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas 

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)
    

In [20]:
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 
shutil.copyfile("CE-T1.nii", "CE-T1_registered.nii")

/home/chansik/MONSTR


'CE-T1_registered.nii'

In [ ]:
workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

!./MONSTR.sh --t1 CE-T1_registered.nii --t2 T2_registered.nii --fl FLAIR_registered.nii --o output --atlasdir TBI_atlas --reg --clean


In [27]:
import numpy as np
import os
import nibabel as nib
#import matplotlib.pyplot as plt
#from matplotlib.image import imsave
#from PIL import Image

workingDir = '/home/chansik/MONSTR'
%cd {workingDir} 

mask_array = nib.load("atlas4_brainmask_ANTS.nii.gz").get_fdata()
    
for file in ["UE-T1_registered.nii", "CE-T1_registered.nii", "T2_registered.nii", "FLAIR_registered.nii", "ADC_registered.nii"]:
    img_array = nib.load(file).get_fdata()
    stripped_array = img_array * mask_array
    
    newName = file.split(".")[0].split("_")[0] + "_stripped.nii"
    
    niftiFile = nib.Nifti1Image(stripped_array, np.eye(4)) 
    nib.save(niftiFile, newName)

/home/chansik/MONSTR


In [24]:
file = "UE-T1_registered.nii"
file.split(".")[0].split("_")[0]

'UE-T1'

In [ ]:
    
    
    # 2. Skull stripping by MONSTR algorithm following 
    #          1) rigid coregistration of T2 onto T1 (ANTs)
    #          2) N4 bias correction (ANTs)
    !./MONSTR.sh --t1 {t1} --t2 {t2} --o output --atlasdir TBI_atlas 

    # 3. Copy the relevant files (brain Mask and processed T1/T2 images)
    createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

    for file in os.listdir(createdDir):
        if file == 'atlas4_brainmask_ANTS.nii.gz':
            newFileName = mrn + '_bMask.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't1.nii.gz':
            newFileName = mrn + '_T1GD_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

        if file == 't2.nii.gz':
            newFileName = mrn + '_T2_processed.nii.gz'
            shutil.copyfile(os.path.join(createdDir, file), os.path.join(targetDir, newFileName))

    # 4. Clean the temporarily created files and directories
    os.remove(os.path.join(workingDir, t1))
    os.remove(os.path.join(workingDir, t2))
    shutil.rmtree(createdDir)

## centering, cropping, and selecting slices

In [1]:
# define a function

def center_crop(img, margin=10):
    # convert an image into an sqaure image if the dimensions of x and y are different
    if img.shape[0] != img.shape[1]:
        if img.shape[0] < img.shape[1]:
            pad_width = int((img.shape[1] - img.shape[0])//2)
            img = np.pad(img, ((pad_width, pad_width), (0,0), (0,0)), constant_values=0)

        if img.shape[0] > img.shape[1]:
            pad_width = int((img.shape[0] - img.shape[1])//2)
            img = np.pad(img, ((0,0), (pad_width, pad_width), (0,0)), constant_values=0)
        
    # calculate the non-zero margins of x and y axes.
    x_min, x_max = 1000, 0
    y_min, y_max = 1000, 0

    for z in range(img.shape[2]):
        for row in range(img.shape[0]):
            x = np.nonzero(img[row, :, z])

            if x[0].size == 0:
                continue

            if x_min > x[0].min():
                x_min = x[0].min()
            if x_max < x[0].max():
                x_max = x[0].max()

        for col in range(img.shape[1]):
            y = np.nonzero(img[:, col, z])

            if y[0].size == 0:
                continue

            if y_min > y[0].min():
                y_min = y[0].min()
            if y_max < y[0].max():
                y_max = y[0].max()

    x_width = x_max - x_min
    x_center = x_min + (x_width//2)
    y_width = y_max - y_min
    y_center = y_min + (y_width//2)

    if x_width >= y_width:
        new_width = x_width + margin
    else: 
        new_width = y_width + margin

    x_start, x_end = x_center-(new_width//2), x_center+(new_width//2)
    y_start, y_end = y_center-(new_width//2), y_center+(new_width//2)

    new_img = img[y_start:y_end, x_start:x_end, :]
    
    return new_img

In [2]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.image import imsave
from PIL import Image

sourceDir = '/media/chansik/SC_research_2019/GBM-Surv/ProcessedByMONSTR'
outputDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/output-multiple'

range_df = pd.read_csv('/home/chansik/DataAnalysis/201912-GBM-Surv/input/range_list.csv', header=None)

target_img_size = (224, 224)

for idx, row in range_df.iterrows():
    t1_name = os.path.join(sourceDir, (str(row[0]) + "_T1GD_processed.nii.gz"))
    t2_name = os.path.join(sourceDir, (str(row[0]) + "_T2_processed.nii.gz"))
    mask_name = os.path.join(sourceDir, (str(row[0]) + "_bMask.nii.gz"))
    
    # Get Image Array (numpy array) from Nifti File
    t1_array = nib.load(t1_name).get_fdata()
    t2_array = nib.load(t2_name).get_fdata()
    mask_array = nib.load(mask_name).get_fdata()
    
    t1_img = t1_array * mask_array
    t1_img = t1_img[:, :, int(row[1]):int(row[2])]
    
    t2_img = t2_array * mask_array
    t2_img = t2_img[:, :, int(row[1]):int(row[2])]
    
    cropped_t1 = center_crop(t1_img, margin=10) # the function 'center_crop' was defined above
    cropped_t2 = center_crop(t2_img, margin=10)
    
    height = int(row[2]) - int(row[1])
    
    for slc in range(2, height, 5):
        selected_t1 = cropped_t1[:, :, slc]
        selected_t2 = cropped_t2[:, :, slc]
        
        newT1name = os.path.join(outputDir, (str(row[0]) + "_T1_" + str(slc) + ".png"))
        newT2name = os.path.join(outputDir, (str(row[0]) + "_T2_" + str(slc) + ".png"))

        imsave(newT1name, selected_t1, cmap='gray')
        imsave(newT2name, selected_t2, cmap='gray')

## resizing and stacking

In [45]:
import pandas as pd
import numpy as np
import os
import shutil
import random

baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
sourceDir = os.path.join(baseDir, 'output-multiple')
targetDir = os.path.join(baseDir, 'cnn_input')

df = pd.read_csv(os.path.join(baseDir, 'input/GBM-surv-list.csv'))
df = df.loc[df.death==1, ['ID', 'survival']]

short = df.loc[df.survival < df.survival.median(), 'ID'].tolist()
long = df.loc[df.survival >= df.survival.median(), 'ID'].tolist()

short_validation_list = random.choices(short, k=30)
long_validation_list = random.choices(long, k=30)

for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])
    
    if mrn in short:
        if mrn in short_validation_list:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation/short', file))
        else:
             shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train/short', file))
    
    if mrn in long:
        if mrn in long_validation_list:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation/long', file))
        else:
             shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train/long', file))

In [ ]:
for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])
    
    if mrn in df.ID.tolist():
        survival = df.loc[df.ID==mrn, 'survival'].values[0]
        new_fileName = str(survival) + "_" + str(mrn) + ".png"

        if mrn in validation_set:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'validation', new_fileName))
        else:
            shutil.copyfile(os.path.join(sourceDir, file), 
                            os.path.join(targetDir, 'train', new_fileName))
    

In [ ]:
baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
sourceDir = os.path.join(baseDir, 'output')
targetDir = os.path.join(baseDir, 'cnn_regression')

[p.split('_')[os.listdir(targetDir)

### divide patients into two: short and long

In [ ]:

import pandas as pd
import numpy as np
import os
import shutil

df = pd.read_csv('/home/chansik/DataAnalysis/201912-GBM-Surv/input/GBM-surv-list.csv')
df = df.loc[df.death==1, ['ID', 'survival']]

short = df.loc[df.survival < df.survival.median(), 'ID'].tolist()
long = df.loc[df.survival >= df.survival.median(), 'ID'].tolist()

sourceDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/output'
shortDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/cnn_input/short'
longDir = '/home/chansik/DataAnalysis/201912-GBM-Surv/cnn_input/long'

for file in os.listdir(sourceDir):
    
    # patient ID number (MRN) is extracted as a variable 'mrn'
    mrn = int(file.split('_', 1)[0])

    if mrn in short:
        shutil.copyfile(os.path.join(sourceDir, file), os.path.join(shortDir, file))
    if mrn in long:
        shutil.copyfile(os.path.join(sourceDir, file), os.path.join(longDir, file))

## convert gray to RGB

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import tensorflow as tf
from matplotlib.image import imsave

baseDir = '/home/chansik/DataAnalysis/201912-GBM-Surv'
print(os.path.join(baseDir, 'output'))
for file in os.listdir(os.path.join(baseDir, 'output')):
    final_img_name = os.path.join(baseDir, 'gray-to-rgb', file)
    converted = tf.image.grayscale_to_rgb(file, name=None)
    imsave(final_img_name, converted)
    


In [ ]:
from PIL import Image
resized_image = Image.fromarray(new_img[:,:,20]).resize((224, 224))

nifti_file = nib.Nifti1Image(new_img, np.eye(4))
nib.save(nifti_file, 'test.nii') # Save

In [ ]:
plt.imshow(np.array(resized_image))

In [ ]:
plt.imshow(img[:, :, 29])

In [ ]:
print(img.shape)
new = np.pad(img, pad_width = ((1,1), (0,0), (0,0)), constant_values=0)
print(new.shape)

In [ ]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.image import imsave
from PIL import Image

image_array = nib.load("251020_T1GD_processed.nii.gz").get_fdata()
mask_array = nib.load("251020_bMask.nii.gz").get_fdata()
img = image_array * mask_array
img = img[:, :, 53:113]
 